# Quartal Mittelwert
_Weitere Informationen sind im Code in den mit # markierten Zeilen zu finden_

## Die Imports

In [24]:
# IBM Cloud
from ibm_botocore.client import Config
import ibm_boto3


import csv
import time
import datetime

## Spezifikation für das Funktionieren in der Cloud
_Die folgenden Codefelder werden nur im bereich der Cloud gebraucht und entfallen bei benutzung auf einem lokalen System_

An den Stellen an welchen die Punkte '.........' sind werden eigentlich die erforderlichen Infos eingesetzt, welche aber aus sicherheitstechnischen Gründen von mir entfernt wurden 

In [25]:
# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
cosCreds = {
    'IBM_API_KEY_ID': '........',
    'IAM_SERVICE_ID': 'crn:v1:bluemix:public:iam-identity::a/a6c0651c3aab4e74a4d1e131c1858b21::serviceid:ServiceId-cf18b8e2-291a-4175-b02f-4fa012271ab7',
    'ENDPOINT': 'https://s3.private.eu-de.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': '........',
    'BUCKET': '........'
}

#### Aus der Cloud lesen

In [26]:
def read_file_cos(credentials,local_file_name, key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    data_cos = None 
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded: %s' %(key))

### Die Angaben

In [28]:

year    = 2018              #year          
sensor  =  299              #sensor
quartal =    4              #quartal


# die Monate eines Quartals 
qMonths = []

## Der eigentliche Code

In [29]:
# Start der Zeit für die Zeitmessung
start = time.time()

# das aktuelle Datum
now = datetime.datetime.now()


# definierung des Quartals
if quartal == 4:
    months = [10,11,12]
    qMonths.extend(months)

elif quartal == 3:
    months = (7,8,9)
    qMonths.extend(months)

elif quartal == 2:
    months = (4,5,6)
    qMonths.extend(months)

elif quartal == 1:
    months = (1,2,3)
    qMonths.extend(months)
else:
    print("invalid quartal")

sPM2_5 = []
sPM10 = []

# Anzahl der PM2_5 und PM10 Werte
anzAllPM10 = 0
anzAllPM2_5 = 0

# Summe aller addierten PM10 Werte
wertAllPM10 = 0

# Summe aller addierten PM2_5 Werte
wertAllPM2_5 = 0

# das herraussuchen der files in einem Quartal
for y in [year]:
    for m in qMonths:
        for d in range(1,32):
            if m in [4,6,9,11] and d > 30:
                continue
            if m == 2 and d >28:
                continue
            # stop yesterday
            if y == 2019 and m > now.month:
                continue 
            if y == 2019 and m == now.month and d > now.day -1:
                continue
                
            filename = "%04d-%02d-%02d_sds011_sensor_%d.csv" %(y,m,d,sensor)
                        
    
            localPath = "/var/tmp/%s" %(filename)
            
            read_file_cos(cosCreds,localPath,filename)

            try:
                with open(localPath,newline = '') as f:
                    readers = csv.DictReader(f, delimiter= ';')
                    for row in readers:
                        PM10 = float(row["P1"])
                        PM2_5 = float(row["P2"])
                        sPM10.append(PM10)
                        sPM2_5.append(PM2_5)
                        
                
                    
            except:
                print("file %04d-%02d-%02d is not avalible" %(y,m,d))
                

for i in range(0, len(sPM10)):

    #PM10 
    wertAllPM10 += sPM10[i]
    anzAllPM10 += 1



for i in range(0, len(sPM2_5)):
    
    #PM2_5
    wertAllPM2_5 += sPM2_5[i]
    anzAllPM2_5 += 1                

           
# Durchschnittswert = Summe aller Werte durch Anzahl aller Werte
durchWertPM10 = wertAllPM10 / anzAllPM10
durchWertPM2_5 = wertAllPM2_5 / anzAllPM2_5

# Ende der Zeit für die Zeitmessung
end = time.time()

# endzeit - Anfangszeit = benötigte Zeit
diff = end-start


File Downloaded: 2018-10-01_sds011_sensor_299.csv
File Downloaded: 2018-10-02_sds011_sensor_299.csv
File Downloaded: 2018-10-03_sds011_sensor_299.csv
File Downloaded: 2018-10-04_sds011_sensor_299.csv
File Downloaded: 2018-10-05_sds011_sensor_299.csv
File Downloaded: 2018-10-06_sds011_sensor_299.csv
File Downloaded: 2018-10-07_sds011_sensor_299.csv
File Downloaded: 2018-10-08_sds011_sensor_299.csv
File Downloaded: 2018-10-09_sds011_sensor_299.csv
File Downloaded: 2018-10-10_sds011_sensor_299.csv
File Downloaded: 2018-10-11_sds011_sensor_299.csv
File Downloaded: 2018-10-12_sds011_sensor_299.csv
File Downloaded: 2018-10-13_sds011_sensor_299.csv
File Downloaded: 2018-10-14_sds011_sensor_299.csv
File Downloaded: 2018-10-15_sds011_sensor_299.csv
File Downloaded: 2018-10-16_sds011_sensor_299.csv
File Downloaded: 2018-10-17_sds011_sensor_299.csv
File Downloaded: 2018-10-18_sds011_sensor_299.csv
File Downloaded: 2018-10-19_sds011_sensor_299.csv
File Downloaded: 2018-10-20_sds011_sensor_299.csv


## Die Printouts

In [30]:
print("Der Feinstaubwert (PM10) lag im %d. Quartal des Jahres %d bei %.2f μm pro m^3" 
%(quartal,year, durchWertPM10))
print("Der Feinstaubwert (PM2_5) lag im %d. Quartal des Jahres %d bei %.2f μm pro m^3" 
%(quartal,year, durchWertPM2_5))

print("Sec: %.4f" %(diff)) 

Der Feinstaubwert (PM10) lag im 4. Quartal des Jahres 2018 bei 10.32 μm pro m^3
Der Feinstaubwert (PM2_5) lag im 4. Quartal des Jahres 2018 bei 7.33 μm pro m^3
Sec: 25.5935
